In [55]:
from requests_html import HTMLSession
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


df = pd.DataFrame()
url = "https://www.clinicaltrialsregister.eu/"
s = HTMLSession()

page = s.get(url+ 'ctr-search/search?query=&page=')
last_page_num = 2 # max([int(a.split('=')[-1]) for a in page.html.links if 'page' in a]) 

for page_num in range(1,last_page_num):
    page = s.get(url+ f'ctr-search/search?query=&page={page_num}')
    studies_links = [link for link in page.html.links if link.startswith('/ctr-search/trial') and 'results' not in link]

    
    for study_link in studies_links:
        study = s.get(url+ study_link)
        #Study Summary
        sum_field = study.html.find('td.cellGrey')                                      #All summary fields
        sum_field_value = study.html.find('td.cellLighterGrey')                         #All summary values
        study_info= {x.text:y.text for x,y in zip(sum_field,sum_field_value)} 

        #Study Details
        details = study.html.find('tr.tricell')                                         # List of all the other filed and values (objects)
        for detail in details:
            detail_list = detail.text.split('\n')                                       # Get text of all objects and split to get a list for each filed,value
            if len(detail_list) < 3:
                detail_list.append('')                                                  # Add blanks for fields without values 
            study_info[detail_list[1]] = ' '. join(detail_list[2:])                   # Creates dict with Filed and Value(some filds have multiple values, must clean later if needed)

        df = df.append(study_info, ignore_index=True)
df.to_csv('EUCTRegister.csv', index=False)